# Arithmetik

Eine wichtige Funktion von pandas ist das arithmetische Verhalten bei Objekten mit unterschiedlichen Indizes. Wenn beim Addieren von Objekten die Indexpaare nicht gleich sind, wird der entsprechende Index im Ergebnis die Vereinigung der Indexpaare sein. Für Benutzer mit Datenbankerfahrung ist dies vergleichbar mit einem automatischen [OUTER JOIN](https://de.wikipedia.org/wiki/Join_(SQL)#%C3%84u%C3%9Ferer_Verbund_(OUTER_JOIN)) auf den Indexbezeichnungen. Schauen wir uns ein Beispiel an:

In [1]:
import numpy as np
import pandas as pd


rng = np.random.default_rng()
s1 = pd.Series(rng.normal(size=5))
s2 = pd.Series(rng.normal(size=7))

Addiert man diese Werte, erhält man:

In [2]:
s1 + s2

0    0.544607
1    0.736780
2    1.263293
3    1.126592
4    1.869919
5         NaN
6         NaN
dtype: float64

Der interne Datenabgleich führt zu fehlenden Werten an den Stellen der Labels, die sich nicht überschneiden. Fehlende Werte werden dann bei weiteren arithmetischen Berechnungen weitergegeben.

Bei DataFrames wird die Ausrichtung sowohl für die Zeilen als auch für die Spalten durchgeführt:

In [3]:
df1 = pd.DataFrame(rng.normal(size=(5,3)))
df2 = pd.DataFrame(rng.normal(size=(7,2)))

Wenn die beiden DataFrames addiert werden, ergibt sich ein DataFrame, dessen Index und Spalten die Vereinigungen derjenigen in jedem der obigen DataFrames sind:

In [4]:
df1 + df2

,0,1,2
0,0.747042,0.422536,NaN
1,0.129950,2.830772,NaN
2,2.900847,-2.108385,NaN
3,4.163107,0.842881,NaN
4,0.133227,-0.234049,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN


Da die Spalte 2 nicht in beiden DataFrame-Objekten vorkommen, erscheinen sie im Ergebnis als fehlend. Das Gleiche gilt für die Zeilen, deren Bezeichnungen nicht in beiden Objekten vorkommen.

## Arithmetische Methoden mit Füllwerten

Bei arithmetischen Operationen zwischen unterschiedlich indizierten Objekten kann es sinnvoll sein, einen speziellen Wert (z. B. `0`) zu verwenden, wenn eine Achsenbeschriftung in einem Objekt gefunden wird, im anderen aber nicht.  Mit der `add`-Methode kann das Argument `fill_value` übergeben werden:

In [5]:
df12 = df1.add(df2, fill_value=0)

df12

,0,1,2
0,0.747042,0.422536,-2.642356
1,0.129950,2.830772,-0.795261
2,2.900847,-2.108385,1.689601
3,4.163107,0.842881,-0.235145
4,0.133227,-0.234049,-0.969567
5,0.678867,0.579447,NaN
6,-0.220217,-2.023013,NaN


Im folgenden Beispiel setzen wir die beiden verbleibenden `NaN`-Werte auf `0`:

In [6]:
df12.iloc[[5,6], [2]] = 0

In [7]:
df12

,0,1,2
0,0.747042,0.422536,-2.642356
1,0.129950,2.830772,-0.795261
2,2.900847,-2.108385,1.689601
3,4.163107,0.842881,-0.235145
4,0.133227,-0.234049,-0.969567
5,0.678867,0.579447,0.000000
6,-0.220217,-2.023013,0.000000


## Arithmetische Methoden

Methode | Beschreibung
:------ | :-----------
`add`, `radd` | Methoden für Addition (`+`)
`sub`, `rsub` | Methoden für die Subtraktion (`-`)
`div`, `rdiv` | Methoden für die Division (`/`)
`floordiv`, `rfloordiv` | Methoden für die Abrundungsfunktion (engl.: floor divison) (`//`)
`mul`, `rmul` | Methoden für die Multiplikation (`*`)
`pow`, `rpow` | Methoden zur Potenzierung (`**`)

`r` (engl.: _reverse_) kehrt die Methode um.

## Operationen zwischen DataFrame und Series

Wie bei NumPy-Arrays verschiedener Dimensionen ist auch die Arithmetik zwischen DataFrame und Series definiert.

In [8]:
s1 + df12

,0,1,2,3,4
0,0.692009,0.116961,-1.090685,NaN,NaN
1,0.074917,2.525197,0.756409,NaN,NaN
2,2.845814,-2.413960,3.241272,NaN,NaN
3,4.108074,0.537307,1.316525,NaN,NaN
4,0.078194,-0.539624,0.582104,NaN,NaN
5,0.623834,0.273872,1.551671,NaN,NaN
6,-0.275250,-2.328588,1.551671,NaN,NaN


Wenn wir `s1` mit `df12` addieren, wird die Additon für jede Zeile einmal durchgeführt. Dies wird als _Broadcasting_ bezeichnet. Standardmäßig entspricht die Arithmetik zwischen DataFrame und Serie dem Index der Serie in den Spalten des DataFrame, wobei die Zeilen nach unten übertragen werden.

Wenn ein Indexwert weder in den Spalten des DataFrame noch im Index der Serie gefunden wird, werden die Objekte neu indiziert, um die Vereinigung zu bilden:

Wenn ihr stattdessen die Spalten übertragen und die Zeilen abgleichen wollt, müsst ihr eine der arithmetischen Methoden verwenden, z.B.:

In [9]:
df12.add(s2, axis='index')

,0,1,2
0,1.346683,1.022177,-2.042716
1,1.172305,3.873127,0.247094
2,2.612469,-2.396763,1.401223
3,5.827087,2.506862,1.428835
4,1.926516,1.559240,0.823722
5,-0.946801,-1.046222,-1.625668
6,-2.305034,-4.107830,-2.084817


Die Achsennummer, die ihr übergebt, ist die Achse, auf die abgeglichen werden soll. In diesem Fall soll der Zeilenindex des DataFrame (`axis='index'` oder `axis=0`) abgeglichen und übertragen werden.

## Funktionsanwendung und Mapping

`numpy.ufunc` (elementweise Array-Methoden) funktionieren auch mit Pandas-Objekten:

In [10]:
np.abs(df12)

,0,1,2
0,0.747042,0.422536,2.642356
1,0.129950,2.830772,0.795261
2,2.900847,2.108385,1.689601
3,4.163107,0.842881,0.235145
4,0.133227,0.234049,0.969567
5,0.678867,0.579447,0.000000
6,0.220217,2.023013,0.000000


Eine weitere häufige Operation ist die Anwendung einer Funktion auf eindimensionale Arrays auf jede Spalte oder Zeile. Die [pandas.DataFrame.apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)-Methode tut genau dies:

In [11]:
df12

,0,1,2
0,0.747042,0.422536,-2.642356
1,0.129950,2.830772,-0.795261
2,2.900847,-2.108385,1.689601
3,4.163107,0.842881,-0.235145
4,0.133227,-0.234049,-0.969567
5,0.678867,0.579447,0.000000
6,-0.220217,-2.023013,0.000000


In [12]:
f = lambda x: x.max() - x.min()

df12.apply(f)

0    4.383323
1    4.939157
2    4.331957
dtype: float64

Hier wird die Funktion `f`, die die Differenz zwischen dem Maximum und dem Minimum einer Reihe berechnet, einmal für jede Spalte des Rahmens aufgerufen. Das Ergebnis ist eine Reihe mit den Spalten des Rahmens als Index.

Wenn ihr `axis='columns'` an `apply` übergebt, wird die Funktion stattdessen einmal pro Zeile aufgerufen:

In [13]:
df12.apply(f, axis='columns')

0    3.389398
1    3.626034
2    5.009232
3    4.398252
4    1.102794
5    0.678867
6    2.023013
dtype: float64

Viele der gebräuchlichsten Array-Statistiken (wie `sum` und `mean`) sind DataFrame-Methoden, so dass die Verwendung von `apply` nicht notwendig ist.

Die an `apply` übergebene Funktion muss keinen Einzelwert zurückgeben; sie kann auch eine Reihe mit mehreren Werten zurückgeben:

In [14]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])

df12.apply(f)

,0,1,2
min,-0.220217,-2.108385,-2.642356
max,4.163107,2.830772,1.689601


Es können auch elementweise Python-Funktionen verwendet werden. Angenommen, ihr möchtet jeden Fließkommawert in `df12` auf zwei Nachkommastellen runden, dann könnt ihr dies mit [pandas.DataFrame.applymap](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.applymap.html) erreichen:

In [15]:
f = lambda x: round(x, 2)

df12.applymap(f)

,0,1,2
0,0.75,0.42,-2.64
1,0.13,2.83,-0.80
2,2.90,-2.11,1.69
3,4.16,0.84,-0.24
4,0.13,-0.23,-0.97
5,0.68,0.58,0.00
6,-0.22,-2.02,0.00


Der Grund für den Namen `applymap` ist, dass Series über eine `map`-Methode zur Anwendung einer elementweisen Funktion verfügt:

In [16]:
df12[2].map(f)

0   -2.64
1   -0.80
2    1.69
3   -0.24
4   -0.97
5    0.00
6    0.00
Name: 2, dtype: float64